Пространство кватернионов.
Заметка: деление кватернионов осуществляется умножением на обратный кватернион, который находится следующим образом:
$$ q^{-1} = \frac{\bar q}{{|q|}^{2}} $$
Где $|q|$ - это модуль делителя, а $\bar q$ - его сопряженное

In [37]:
from numbers import Number
from math import sqrt
import itertools


class QuaterniDomainError(ValueError):
    pass


class Quaterni:
    
    def __init__(self, arg=0):
        if isinstance(arg, Number):
            self.coefficients = [arg]
        elif isinstance(arg, list):
            self.coefficients = arg.copy()
        elif isinstance(arg, Quaterni):
            self.coefficients = arg.coefficients.copy()
        else:
            raise QuaterniDomainError("You are trying to create quaterni from " + repr(arg))
        if len(self.coefficients) > 4:
            for _ in range(len(self.coefficients) - 4): #Если список больше 4, то просто удаляю лишнее
                del self.coefficients[-1]
        self.zero_delete(self.coefficients)


    @staticmethod
    def zero_delete(coefficients):
        """
        Удаляю лишние нули. Если список будет например [1, 0, 0, 0, 0, 0, 0], то он
        будет преобразован в список [1]. Если список [0, 0, 0], то список -> []
        """
        while len(coefficients) > 0 and coefficients[-1] == 0:
            del coefficients[-1]


    def __str__(self):
        """
        Немного модернизировал так, чтобы не показывались слагаемые с пустыми коэффициентами
        """
        return (" + ".join([
            str(c) + ("*" + i if i != '' else '')
            for c, i in list(zip(self.coefficients, ['','i', 'j', 'k'])) if c != 0
            ])) if len(self.coefficients) and sum(self.coefficients) else '0'


    def __eq__(self, other):
        if isinstance(other, Number):
            other = Quaterni(other)
        
        if isinstance(other, Quaterni):
            return self.coefficients == other.coefficients
        else:
            raise QuaterniDomainError("Can't say if Quaterni is equal to " + str(type(other)))


    def __lshift__(self, deg):
        return Quaterni(([0] * deg) + self.coefficients)


    def __add__(self, other):
        if isinstance(other, Number):
            other = Quaterni(other)
        
        sc = self.coefficients.copy()
        oc = other.coefficients.copy()

        sc += [0] * (4-len(sc))
        oc += [0] * (4-len(oc))
        
        return Quaterni([
            sce + oce for sce, oce in zip(sc, oc)
        ])


    def __radd__(self, other):
        return self.__add__(other)


    def __neg__(self):
        return Quaterni([-c for c in self.coefficients])


    def __sub__(self, other):
        if isinstance(other, Number):
            other = Quaterni(other)

        return self.__add__(other.__neg__())


    def __rsub__(self, other):
        return self.__neg__().__add__(other)


    def __mul__(self, other):
        """
        Эффективный способ умножение кватернионов подобрать было сложно,
        так умножение не коммутативно, поэтому я применил "прямую" формулу
        """
        if isinstance(other, Number):
            other = Quaterni(other)
        if isinstance(self, Number):
            self = Quaterni(self)
        
        s = self.coefficients.copy() + [0] * (4 - len(self.coefficients))
        o = other.coefficients.copy() + [0] * (4 - len(other.coefficients))
        
        """
        Возможно это не корректно, но я решил округлить результаты умножения, чтобы не возникали
        слишком длинные числа
        """
        return Quaterni([round(s[0] * o[0] - s[1] * o[1] - s[2] * o[2] - s[3] * o[3], 5),
                         round(s[0] * o[1] + o[0] * s[1] + s[2] * o[3] - o[2] * s[3], 5),
                         round(s[0] * o[2] + o[0] * s[2] + s[3] * o[1] - o[3] * s[1], 5),
                         round(s[0] * o[3] + o[0] * s[3] + s[1] * o[2] - o[1] * s[2], 5)])


    def __abs__(self):
        """Моудль"""
        return sqrt(sum(i**2 for i in self.coefficients))


    def __invert__(self):
        """Сопряженное"""
        return Quaterni([self.coefficients[0]] + [-c for c in self.coefficients[1: ]])


    def __truediv__(self, other):
        """Деление, осуществляемое описанным методом"""
        if isinstance(self, Number):
            self = Quaterni(self)
        if isinstance(other, Number):
            other = Quaterni(other)

        if other.coefficients == []:
            raise QuaterniDomainError("Can't divide by zero")
        else:
            return self.__mul__(Quaterni([i/(other.__abs__())**2 for i in other.__invert__().coefficients]))


    def __complex__(self):
        if not len(self.coefficients):
            return 0j
        elif len(self.coefficients) == 1:
            return complex(self.coefficients[0])
        elif len(self.coefficients) == 2:
            return complex(self.coefficients[0], self.coefficients[1])
        else:
            raise QuaterniDomainError("Quaternion cannot be converted to a complex number")
        

    def __float__(self):
        if not len(self.coefficients):
            return 0.0
        elif len(self.coefficients) == 1:
            return float(self.coefficients[0])
        else:
            raise QuaterniDomainError("Quaternion cannot be converted to a floating point number")
        

    def __int__(self) -> int:
        if not len(self.coefficients):
            return 0
        elif len(self.coefficients) == 1:
            return int(self.coefficients[0])
        else:
            raise QuaterniDomainError("Quaternion cannot be converted to a integer number")
        
        return int(self.coefficients[0])

In [41]:
q1 = Quaterni([5, 0, 7, 2, 9, 0, 6, 9])
q2 = Quaterni([0, 3, 0, 6])
q3 = Quaterni([2, 3])
q4 = Quaterni(5)
print(q1)
print(q2)
print(q1 + q2)
print(q1 * q2)
print(abs(q1))
print(~q1)
print(q1/q2)
print(complex(q3))
print(float(q4), int(q4))

5 + 7*j + 2*k
3*i + 6*k
5 + 3*i + 7*j + 8*k
-12 + 57*i + 6*j + 9*k
8.831760866327848
5 + -7*j + -2*k
0.26667 + -1.26667*i + -0.13333*j + -0.2*k
(2+3j)
5.0 5
